In [ ]:
pip install -q yahoo_fin

In [2]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml@v0.0.5#egg=runml

In [4]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

Run following if like to save results in drive

In [5]:
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [6]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accuracy']/stop_loss - (1-df['Accuracy'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
pending = ['PAR']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRCT', 'CRWD',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DXCM',
           'ESTC', 'ETSY', 'EXPE', 'FOUR', 'GFS', 'GLBE', 'GOGO',
           'INDI', 'INMD', 'INTA', 'IOT','JKS', 'LUMN', 'LYFT',
           'MAXR', 'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR','OKTA', 'OLED', 'OSTK',
           'PANW', 'PAYO', 'PD', 'PLUG', 'PI', 'PINS', 'PTON', 'PUBM', 'RBLX',
           'SNAP', 'SNOW', 'SPLK', 'SPOT','STEM', 'STNE','SWAV',
           'TEAM', 'TDOC', 'TOST', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df  = pipeline.runModelCombined(tickers, 'ipos-3b', mod, True, loss=lossfn)[0]

In [ ]:
addAlloc(df, 0.10, 1)
df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Accuracy,Error,Gain,Last,Predicted,Alloc
36,MNDY,RROnly,533.22,1517.63,2050.85,0.962025,83.276822,0.24,142.75,176.940002,9.462025
53,RBLX,RROnly,341.40,416.45,757.85,0.956522,25.599198,0.12,44.98,50.270000,9.202899
12,DASH,RROnly,548.76,1010.02,1558.78,0.942308,47.332274,0.26,63.56,79.870003,9.201183
56,SPLK,RROnly,1200.91,1101.14,2302.05,0.946809,71.120525,0.15,95.88,110.589996,9.113475
15,DOCN,RROnly,298.90,412.19,711.09,0.966667,25.843376,0.06,39.17,41.560001,9.111111
19,ESTC,RROnly,855.83,1168.76,2024.59,0.941489,46.717172,0.19,57.90,68.800003,9.106943
57,SPOT,RROnly,2179.26,1471.20,3650.46,0.946809,75.900741,0.14,133.62,152.550003,9.088146
63,TOST,RROnly,64.24,125.74,189.98,0.937500,13.493710,0.19,17.75,21.170000,9.046053
31,LUMN,RROnly,55.15,89.30,144.45,0.925532,2.645336,0.34,2.65,3.550000,9.036295
50,PINS,RROnly,343.05,450.21,793.26,0.935829,12.193432,0.19,27.27,32.570000,9.020546


In [ ]:
findata.EPOCHS=200
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX', 
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MOAT', 'MTUM', 
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df1, res = pipeline.runModelCombined(tickers1, 'etf-3a', mod, False)
# !cp -r results /content/drive/MyDrive/colab

In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

(0.9202127659574468,
 0.9229203567322334,
 0.09075827015321843,
 0.004420030125699004,
 0.01842253193096099)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Accuracy,Error,Gain,Last,Predicted,Alloc
3,DAPP,RROnly,62.06,69.95,132.01,0.942529,2.880158,0.76,4.49,7.890000,9.349667
18,MGV,RROnly,311.00,259.14,570.14,0.978723,52.968813,-0.04,102.00,98.419998,9.255319
21,ONLN,RROnly,292.16,393.42,685.59,0.941489,26.828001,0.35,30.63,41.250000,9.247720
39,XLU,RROnly,239.08,257.94,497.02,0.952128,41.473744,-0.17,69.81,57.779999,9.239675
2,ARKW,RROnly,717.96,786.74,1504.69,0.941489,37.945955,0.21,51.27,62.049999,9.136272
1,ARKK,RROnly,529.99,697.86,1227.85,0.930851,30.982531,0.32,38.57,51.060001,9.092420
10,IPO,RROnly,304.13,323.31,627.45,0.946809,22.112465,0.14,28.05,31.850000,9.088146
0,ARKF,RROnly,256.42,228.26,484.67,0.930851,14.167429,0.25,18.04,22.480000,9.031915
40,XLV,RROnly,375.14,265.47,640.61,0.941489,72.164278,-0.10,133.52,120.730003,8.829787
32,XLB,RROnly,337.46,221.87,559.33,0.957447,36.692507,-0.05,79.63,76.019997,8.723404


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
# exclude = ['GTSWF', 'CBNT', 'CCL', 'CREX'. 'DTEA'. 'ILAL', 'TNXP', 'MULN', 'AHT',
# 'LILM', 'JMIA', 'RKLB', 'VRAR', 'WISH',]
tickers3 = [ 'AIRI', 'APRN', 'AUGX', 'AUR', 'AMSC', 'ASTR', 'ATOM',
            'BBBY', 'BKKT', 'BGSF', 'BLNK', 'BSGM',
            'CARS', 'CLOV', 'CZOO',
            'DLO', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY', 'FTCH',
            'GOVX', 'GTE', 'HIMS', 'HIVE', 
            'INLX',  'JOBY',  'KIND', 'KULR',
            'LAZR', 'MMMB', 'MRIN', 'MTTR', 
            'NEPH', 'NSTG', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST', 
            'RAD', 'REAL', 'RIDE', 'RIG', 'RVYL', 'SFIX', 'STEM',
            'SFT',  'TELL', 'VERI', 'VLDR',  'YEXT']
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df3 = pipeline.runModelCombined(tickers3, 'vols-4a', mod, True)[0]

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Accuracy,Error,Gain,Last,Predicted,Alloc
40,MULN,RROnly,84.28,111.59,195.87,0.877660,0.392864,-2.06,0.10,-0.11,5.103315
43,OMQS,RROnly,74.89,84.35,159.24,0.920213,3.411185,0.19,4.24,5.05,4.993083
55,STEM,RROnly,140.00,230.13,370.13,0.902655,6.038844,0.26,4.92,6.21,4.935330
8,BBBY,RROnly,244.53,243.81,488.34,0.851064,1.442516,1.12,0.31,0.66,4.873279
60,VRAR,RROnly,25.23,64.27,89.50,0.909091,2.956623,0.19,4.23,5.05,4.869125
1,APRN,RROnly,120.73,121.09,241.82,0.882979,1.027944,0.25,0.68,0.85,4.725907
22,EVGO,RROnly,82.93,100.63,183.56,0.878505,4.128414,0.27,6.51,8.27,4.717692
6,ASTR,RROnly,37.83,105.78,143.60,0.831858,0.801041,0.58,0.37,0.59,4.603385
15,CLOV,RROnly,52.41,76.33,128.75,0.861538,1.170564,0.27,0.83,1.06,4.555053
56,SFT,RROnly,320.37,549.28,869.65,0.790055,3.999609,0.89,1.21,2.29,4.411491


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX', 'TMO', 'TSLA', 'TXN', 'UNH', 'UPS']
mod4 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df4 = pipeline.runModelCombined(tickers4, 'mcap-3a', mod4, False)[0]

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
6,BA,RROnly,1459.73,1682.14,3141.87,99.877690,0.920213,201.71,242.729996,0.20,6.174012
23,META,RROnly,1564.83,1640.74,3205.57,94.503539,0.920213,221.49,247.929993,0.12,5.908055
18,FDX,RROnly,1220.17,1356.91,2577.08,89.923641,0.925532,230.69,251.179993,0.09,5.783519
34,TSLA,RROnly,2029.85,1938.94,3968.79,18.312280,0.877660,185.00,146.220001,-0.21,5.686424
0,AAPL,RROnly,941.80,491.31,1433.11,44.169612,0.914894,165.21,148.539993,-0.10,5.683891
11,CRM,RROnly,1261.81,1314.89,2576.70,127.793893,0.898936,194.65,219.610001,0.13,5.643559
30,PYPL,RROnly,1166.52,1188.72,2355.24,71.723916,0.867021,76.53,92.180000,0.20,5.528116
26,NKE,RROnly,706.74,548.42,1255.15,62.973585,0.909574,125.95,136.910004,0.09,5.492232
12,COST,RROnly,1983.69,1708.54,3692.22,232.847783,0.914894,491.30,451.239990,-0.08,5.471125
4,AMZN,RROnly,916.15,814.62,1730.77,84.030775,0.898936,102.51,113.269997,0.10,5.410334


In [ ]:
findata.EPOCHS=200
tickers5 = ['ABB', 'AEXAF', 'ALSMY', 'AMSSY', 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX',
            'DBOEY', 'DPSGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY', 'EQNR', 'ERIC', 'ERJ','ESEA',
            'FANUY', 'FQVLF', 'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA', 
            'OZON', 'RACE', 'LYB', 'LVMUY',
            'MYTE', 'NHYDY', 'NOK', 'NU', 'NXPI', 'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'VWAPY', 'YARIY', 'ZURVY']
mod5 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df5 = pipeline.runModelCombined(tickers5, 'gstock-4a', mod5, True)[0]


In [ ]:
addAlloc(df5, 0.14)
df5.sort_values('Alloc', ascending=False).head(30)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
43,NU,RROnly,15.70,24.24,39.94,3.514693,0.945455,4.66,5.180000,0.11,5.761511
6,ADYEY,RROnly,153.41,114.85,268.26,6.913049,0.921212,15.88,13.520000,-0.15,5.529582
62,TKAMY,RROnly,77.06,111.60,188.66,4.034997,0.909574,7.84,6.550000,-0.16,5.366641
7,BAK,RROnly,134.79,173.19,307.98,3.851707,0.936170,8.43,7.720000,-0.08,5.091185
29,HDELY,RROnly,86.57,76.02,162.59,5.680474,0.898936,15.22,13.110000,-0.14,4.977204
34,MELI,RROnly,10715.96,7782.55,18498.51,478.246979,0.909574,1301.88,1443.969971,0.11,4.852860
18,EIFZF,RROnly,186.07,168.37,354.43,7.878388,0.877660,39.47,32.639999,-0.17,4.829698
13,CPG,RROnly,51.80,40.83,92.64,0.778576,0.861702,7.78,6.360000,-0.18,4.618372
65,VALE,RROnly,108.08,95.10,203.18,5.317946,0.920213,15.96,14.620000,-0.08,4.578267
60,TELNF,RROnly,39.08,80.32,119.40,8.739421,0.893617,12.39,11.020000,-0.11,4.448743


In [ ]:
findata.EPOCHS=200
tickers6 = ['ARGT', 'EWD', 'CUBS', 'FLN', 'FM',
            'ECH', 'EWA', 'EWI', 'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
            'IDX', 'ILF', 'KEMQ', 'THD', 'TUR', 'XCEM']
mod6 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df6 = pipeline.runModelCombined(tickers6, 'cetf-3b', mod6, False)[0]


In [ ]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
3,FLN,RROnly,78.04,90.13,168.17,9.110142,0.920213,17.62,15.300000,-0.13,8.588380
18,TUR,RROnly,125.99,139.57,265.55,16.973320,0.882979,33.53,24.049999,-0.28,8.411854
16,KEMQ,RROnly,116.51,130.76,247.27,11.923338,0.920213,14.54,15.800000,0.09,8.315603
14,IDX,RROnly,60.32,85.84,146.16,10.163429,0.898936,17.81,15.410000,-0.13,8.211948
9,EWM,RROnly,65.48,86.15,151.63,17.330045,0.941489,21.94,21.040001,-0.04,7.952128
15,ILF,RROnly,110.74,120.80,231.54,12.922415,0.893617,24.11,21.809999,-0.10,7.872340
6,EWA,RROnly,93.43,82.74,176.17,11.637065,0.904255,23.25,21.730000,-0.07,7.674772
1,EWD,RROnly,175.15,147.88,323.03,20.012249,0.904255,36.53,34.110001,-0.07,7.674772
5,ECH,RROnly,157.77,146.79,304.56,16.140604,0.882979,27.56,25.950001,-0.06,6.879433
2,CUBS,RROnly,24.33,33.48,57.81,18.743984,0.925926,19.27,19.850000,0.03,6.790123


In [ ]:
findata.EPOCHS=200
review = ['AMR', 'IFF', 'BLK']
tickers7 = ['AAL', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BGFV', 'BXC',
            'CBT', 'CC', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI',
            'CG',  'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DVN', 'EMN', 
            'F', 'FCX', 'FIS', 'FLO', 'FMS', 'GLW', 'GWW',
            'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN', 'KD', 'KHC', 'KMB', 
             'M', 'MMM', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK', 
            'PARA', 'PCAR', 'PETS', 'PNR', 'PSTL', 'PXD', 'RCL', 'RH',
            'SAFE', 'SCCO', 'SPTN', 'SPG', 'SWK',
            'T', 'TCN', 'TGT', 'TDG', 'TWI', 'UAL', 'UBA', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']
           
mod7 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df7 = pipeline.runModelCombined(tickers7, 'val-4b', mod7, True)[0]

In [ ]:
addAlloc(df7, 0.10, 1)
df7.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
62,TCN,RROnly,20.58,35.97,56.55,7.324075,0.984615,8.06,8.270000,0.03,9.333333
40,KMB,RROnly,500.74,426.51,927.25,102.622699,0.952128,141.57,129.369995,-0.09,8.989362
0,AAL,RROnly,130.60,211.94,342.54,9.525769,0.920213,13.40,15.560000,0.16,8.703457
35,IPI,RROnly,369.95,411.98,781.93,8.412834,0.893617,25.80,16.780001,-0.35,8.632219
71,WDC,RROnly,334.87,458.95,793.82,29.901293,0.920213,33.44,37.860001,0.13,8.588380
8,BXC,RROnly,534.63,526.16,1060.79,5.382333,0.904255,72.28,87.370003,0.21,8.586626
66,UAL,RROnly,421.27,527.20,948.47,21.261948,0.898936,44.45,54.480000,0.23,8.549954
29,FMS,RROnly,141.30,164.30,305.60,13.315466,0.930851,22.85,24.809999,0.09,8.540189
51,PNR,RROnly,254.95,237.55,492.50,22.556911,0.930851,53.46,57.259998,0.07,8.320669
46,OLN,RROnly,403.00,309.90,712.91,9.628269,0.936170,55.62,52.009998,-0.06,8.297872


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'GLD', 'ISHG', 'JJU', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']
mod8 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df8 = pipeline.runModelCombined(tickers8, 'com-3b', mod8, False)[0]

In [8]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
3,PDBC,RROnly,41.60,30.69,72.29,6.680006,0.781915,14.15,11.650000,-0.18,6.607565
10,WEAT,RROnly,41.89,28.99,70.88,5.048091,0.819149,6.38,5.860000,-0.08,5.930851
0,GLD,RROnly,605.72,303.77,909.49,121.886678,0.920213,184.80,181.839996,-0.02,5.212766
2,JJU,RROnly,175.66,143.09,318.74,33.340995,0.718085,48.83,44.169998,-0.10,4.361702
5,SOYB,RROnly,69.54,27.38,96.92,13.637069,0.760638,26.32,27.459999,0.04,1.622340
4,SLV,RROnly,119.25,87.59,206.84,11.590720,0.920213,23.00,23.270000,0.01,1.223404
1,ISHG,RROnly,81.58,82.31,163.89,63.760141,0.856383,70.71,70.470001,-0.00,0.000000
6,TIP,RROnly,120.72,100.24,220.95,97.193149,0.867021,110.18,111.690002,0.01,0.000000
7,TLT,RROnly,362.40,350.52,712.92,92.404694,0.893617,106.46,108.019997,0.01,0.000000
8,UUP,RROnly,37.40,28.91,66.31,24.006243,0.877660,27.75,28.100000,0.01,0.000000


In [ ]:
findata.EPOCHS=200
tickers9 = [ 'ADA-USD', 'BTC-USD', 'DOT-USD', 'DOGE-USD',
            'ETH-USD', 'LTC-USD', 'MATIC-USD', 'XRP-USD']
mod9 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df9 = pipeline.runModelCombined(tickers9, 'cry-4b', mod9, True)[0]

In [10]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
4,ETH-USD,RROnly,22809.58,28025.39,50834.98,450.931806,0.882979,1843.17,2305.310059,0.25,5.838906
5,LTC-USD,RROnly,1906.20,1903.59,3809.79,50.823719,0.840426,87.50,110.690002,0.26,5.389292
0,ADA-USD,RROnly,16.99,8.43,25.42,0.143972,0.776596,0.39,0.640000,0.64,5.198043
3,DOGE-USD,RROnly,2.01,2.18,4.18,0.014245,0.776596,0.08,0.060000,-0.21,4.483283
1,BTC-USD,RROnly,321905.90,313441.67,635347.57,11704.143512,0.824468,28418.29,31554.019531,0.11,4.293313
2,DOT-USD,RROnly,229.69,260.80,490.49,4.431682,0.675676,5.83,9.050000,0.55,4.236574
6,MATIC-USD,RROnly,16.04,14.75,30.79,0.091829,0.856383,0.98,1.040000,0.06,3.723404
7,XRP-USD,RROnly,8.04,8.69,16.73,0.265914,0.728723,0.47,0.480000,0.04,0.000000


In [ ]:
findata.EPOCHS=200
save = ['CF']
tickers10 = ['AAPL', 'AN', 'AZN', 'BJ', 'BLDR',
             'COST', 'CPRX', 'CROX', 'DHR', 'DLTR', 'DY',
             'ELF', 'EQNR',  'HRMY', 'HUM',  'LNTH',
             'MCK',  'MPWR', 'NVDA', 'ON', 'PWR', 'QQQ',  'SWAV', 
             'TSLA', 'UNH', 'URI', 'VRRM', 'XLE']
mod10 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df10 = pipeline.runModelCombined(tickers10, 'trade-8b1', mod10, False)[0]

loading model from results/trade-8b1-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5


In [ ]:
addAlloc(df10, 0.07, 1)
df10.sort_values('Alloc', ascending=False)

In [ ]:
findata.G_LOOKUP_STEP = 5
mod11 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df11 = pipeline.runModelCombined(tickers10, 'trade-8b1', mod11, False)[0]


loading model from results/trade-8b1-sh-1-sc-1-sbd-0-seq-50-step-5-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5


In [ ]:
addAlloc(df11, 0.07, 1)
df11.sort_values('Alloc', ascending=False)

In [11]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
cry-4b-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 3,216,588 bytes  received 35 bytes  6,433,246.00 bytes/sec
total size is 77,099,456  speedup is 23.97
/content/drive/MyDrive/colab/results/com-1b-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/ipos-1b-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
